In [ ]:
#https://github.com/flyyufelix/cnn_finetune/blob/master/vgg19.py

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss


def vgg19_model(img_rows, img_cols, channel=None, num_classes=None):
    """
    VGG 19 Model for Keras
    Model Schema is based on 
    https://gist.github.com/baraldilorenzo/8d096f48a1be4a2d660d
    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilZ2RVeVhKY0FyRmc/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
  
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(img_rows, img_cols,channel)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('vgg19_weights.h5')

    # Truncate and replace softmax layer for transfer learning
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(num_classes, activation='softmax'))

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
#Using Keras VGG19 for transfer learning
import keras
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19

base_model=VGG19(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Adam', loss='categorical_crossentropy')



In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()


In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
import os 
import numpy as np 
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

img_width,img_height = 224, 224
#nb_train_samples =400 
#nb_validation_samples = 100
epochs = 4
batch_size = 16

data_dir = os.path.join(os.getcwd(),'BlobStorage')
validation_data_dir = os.path.join(data_dir, 'validation_data')
train_data_dir = os.path.join(data_dir, 'train_data')


    
train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True) 
  
test_datagen = ImageDataGenerator(rescale=1. / 255) 

f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

#print(lines)

for line in lines:
  
    line = line.split()
   # print(line)
    modelName = line[0]
    classesNow = line[1:]
    print(modelName)
    print(classesNow)
    train_generator = train_datagen.flow_from_directory( 
        train_data_dir, 
        classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 


    validation_generator = test_datagen.flow_from_directory( 
        validation_data_dir, 
        classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 
    n_classes = len(np.unique(train_generator.classes))
    channel = 3

    #model = vgg19_model( img_width,img_height, channel=3, num_classes=n_classes)

    base_model=VGG19(weights='imagenet',include_top=False) 

    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    #x=Dense(1024,activation='relu')(x) #dense layer 2
    #x=Dense(512,activation='relu')(x) #dense layer 3
    preds=Dense(n_classes,activation='softmax')(x) #final layer with softmax activation with n_classes 

    model=Model(inputs=base_model.input,outputs=preds)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional VGG19 layers
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    
    model.fit_generator( 
        train_generator, 
        steps_per_epoch=train_generator.n // batch_size, 
        epochs=epochs, 
        validation_data=validation_generator, 
        validation_steps=validation_generator.n // batch_size)
    modelNameStr = "trained_models/"  + modelName +".h5"
    model.save(modelNameStr)
  

Using TensorFlow backend.


model20
['sunset', 'railway', 'river', 'shoulder', 'bike', 'painting', 'coin', 'coffee', 'championship', 'bag', 'football', 'photograph', 'scene', 'weather', 'shore', 'room', 'car', 'dress', 'roof', 'video']
Found 29905 images belonging to 20 classes.


W0303 01:36:01.183158 140287431530240 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Found 7640 images belonging to 20 classes.


W0303 01:36:01.397815 140287431530240 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0303 01:36:01.469018 140287431530240 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0303 01:36:01.579469 140287431530240 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0303 01:36:06.091310 140287431530240 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0303 01:36:06

Epoch 1/4
 352/1869 [====>.........................] - ETA: 22:28 - loss: 2.4923 - acc: 0.2413

/anaconda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1703936 bytes but only got 0. Skipping tag 42
  " Skipping tag %s" % (size, len(data), tag))
/anaconda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 256000 bytes but only got 0. Skipping tag 19200
  " Skipping tag %s" % (size, len(data), tag))
/anaconda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:768: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1285203968 bytes but only got 0. Skipping tag 59392
  " Skipping tag %s" % (size, len(data), tag))
/anaconda/envs/py36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 9. 
  warnings.warn(str(msg))


1869/1869 [==============================] - 1862s 996ms/step - loss: 2.1760 - acc: 0.3348 - val_loss: 1.9276 - val_acc: 0.4125
Epoch 2/4
1869/1869 [==============================] - 664s 355ms/step - loss: 1.9776 - acc: 0.3959 - val_loss: 1.8282 - val_acc: 0.4332
Epoch 3/4
1869/1869 [==============================] - 663s 355ms/step - loss: 1.9154 - acc: 0.4099 - val_loss: 1.7952 - val_acc: 0.4466
Epoch 4/4
1868/1869 [============================>.] - ETA: 0s - loss: 1.8691 - acc: 0.4256

In [ ]:
#Testing code
from keras.preprocessing.image import ImageDataGenerator 
import os 
import numpy as np 
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

img_width,img_height = 224, 224
#nb_train_samples =400 
#nb_validation_samples = 100
epochs = 5
batch_size = 16

data_dir = os.path.join(os.getcwd(),'BlobStorage')
validation_data_dir = os.path.join(data_dir, 'validation_data')
train_data_dir = os.path.join(data_dir, 'train_data')

train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True) 
  
test_datagen = ImageDataGenerator(rescale=1. / 255) 

f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

#print(lines)

#for line in lines:
line = lines[0]  
print(line)

In [ ]:
line = line.split()
   # print(line)
modelName = line[0]
classesNow = line[1:]
print(modelName)
print(classesNow)
train_generator = train_datagen.flow_from_directory( 
        train_data_dir, 
        classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 


validation_generator = test_datagen.flow_from_directory( 
        validation_data_dir, 
        classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 
n_classes = len(np.unique(train_generator.classes))
channel = 3

    #model = vgg19_model( img_width,img_height, channel=3, num_classes=n_classes)

base_model=VGG19(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x=Dense(1024,activation='relu')(x) #dense layer 2
#x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(n_classes,activation='softmax')(x) #final layer with softmax activation with n_classes 

model=Model(inputs=base_model.input,outputs=preds)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional VGG19 layers
for layer in base_model.layers:
    layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    
model.fit_generator( 
        train_generator, 
        steps_per_epoch=train_generator.n // batch_size, 
        epochs=epochs, 
        validation_data=validation_generator, 
        validation_steps=validation_generator.n // batch_size)
#modelNameStr = "trained_models/"  + modelName +".h5"
#model.save(modelNameStr)

In [ ]:
#!wget --no-check-certificate 'http://files.heuritech.com/weights/vgg19_weights.h5' -O vgg-16_weights.h5
#!wget -O vgg19_weights.h5 https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5

In [ ]:
from keras.models import load_model
import os
img_width,img_height = 224, 224
batch_size = 16
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation


data_dir = os.path.join(os.getcwd(),'BlobStorage')
test_data_dir = os.path.join(data_dir, 'junk') # the categories need to be in folders
print(test_data_dir)

test_datagen = ImageDataGenerator(rescale=1. / 255) 

test_generator = test_datagen.flow_from_directory( 
        test_data_dir, 
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        #class_mode='categorical',
        class_mode=None,
        shuffle = False) 

test_generator.reset()

#Getting list of stored models in trained_models folder
models_list = [l for l in os.listdir("trained_models") if l.endswith('.h5')]
print(models_list)

#model = load_model("model_contextobject_4classes.h5")

filenames =  test_generator.filenames
nb_samples = len(filenames)

print('nb_samples '+str(nb_samples))
print(filenames[1])
#predictions = model.predict_generator(test_generator, steps=nb_samples)

In [ ]:
model = load_model('trained_models/'+models_list[1])
print(models_list[1])
predictions = model.predict_generator(test_generator, steps=nb_samples/batch_size, verbose = 1)
print(predictions)

#predicted_class_indices=np.argmax(pred,axis=1)
#labels = (validation_generator.class_indices)
#labels2 = dict((v,k) for k,v in labels.items())

In [ ]:
predictions.shape


In [ ]:
predicted_class_indices=np.argmax(predictions,axis=1)
labels = (test_generator.class_indices)
labels2 = dict((v,k) for k,v in labels.items())